In [1]:
import os
import glob
import pickle
import numpy as np
import ddsp.training
import tensorboard as tb
import tensorflow as tf
from ddsp import core
from matplotlib import pyplot as plt
import librosa

2024-03-06 18:34:05.823936: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-06 18:34:06.302125: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-03-06 18:34:06.302164: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-03-06 18:34:06.302170: W tensorfl

In [2]:

tf.config.run_functions_eagerly(True)
#tf.data.experimental.enable_debug_mode()
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


2024-03-06 18:34:08.291602: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-06 18:34:08.295170: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-06 18:34:08.295266: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [3]:
MODEL_NAME = 'cello_div' # enter your model name
DATASET_NAME = 'cello_div' # enter your dataset name
NUM_SHARDS = 10
SAMPLE_RATE = 16000
#AUDIO_DIR = '../../data/' + DATASET_NAME + '/audio'
AUDIO_DIR = '../../data/audio/' + DATASET_NAME
AUDIO_FILEPATTERN = AUDIO_DIR + '/*'
SAVE_DIR = '../../data/tfrecords/' + MODEL_NAME
TRAIN_TFRECORD = SAVE_DIR + '/train.tfrecord'
TRAIN_TFRECORD_FILEPATTERN = TRAIN_TFRECORD + '*'
print(SAVE_DIR)

../../data/tfrecords/Matthias


In [4]:
data_provider = ddsp.training.data.TFRecordProvider(TRAIN_TFRECORD_FILEPATTERN, centered=True) # Set 'centered = False' if you are not using center paddding 
dataset = data_provider.get_dataset(shuffle=False)

2024-03-06 18:34:16.635947: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-06 18:34:16.636701: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-06 18:34:16.636826: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-06 18:34:16.636914: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

In [5]:
%reload_ext tensorboard
import tensorboard as tb
tb.notebook.start('--logdir "{}"'.format(SAVE_DIR))

In [6]:
BATCH_SIZE = 16
NUM_STEPS = 500000
STEPS_PER_SAVE = 500
CKPTS_TO_KEEP = 20

In [7]:
MODEL_NAME = 'cello_div'
SAVE_DIR = '../../models/' + MODEL_NAME
print(SAVE_DIR)

../../models/Matthias


In [ ]:
!ddsp_run \
  --mode=train \
  --alsologtostderr \
  --save_dir="$SAVE_DIR" \
  --gin_file=models/solo_instrument.gin \
  --gin_file=datasets/tfrecord.gin \
  --gin_param="TFRecordProvider.file_pattern=\"$TRAIN_TFRECORD_FILEPATTERN\"" \
  --gin_param="batch_size=$BATCH_SIZE" \
  --gin_param="train_util.train.num_steps=$NUM_STEPS" \
  --gin_param="train_util.train.steps_per_save=$STEPS_PER_SAVE" \
  --gin_param="trainers.Trainer.checkpoints_to_keep=$CKPTS_TO_KEEP" \
  --gin_param="TFRecordProvider.centered=True" # Set 'centered = False' if you are not using center paddding 


2024-03-06 18:34:31.339758: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-06 18:34:31.839009: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-03-06 18:34:31.839051: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-03-06 18:34:31.839058: W tensorfl